In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

In [3]:
spark = SparkSession.builder \
    .appName("KafkaStructuredStreaming") \
    .getOrCreate()

In [4]:
spark

In [9]:
kafka_bootstrap_servers = "localhost:9092"
kafka_read_topic = "topic1"
kafka_write_topic = "topic2"
schema = "value STRING"
checkpoint_location = "./kafka_checkpoints"

In [6]:
streaming_df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", kafka_bootstrap_servers) \
    .option("subscribe", kafka_read_topic) \
    .load() \
    .selectExpr("CAST(value AS STRING)")

In [ ]:
streaming_df.printSchema()

In [14]:
words_df = streaming_df.select(
    explode(split(streaming_df.value, " ")).alias("value")
)

In [16]:
words_df.printSchema()

root
 |-- value: string (nullable = false)



In [17]:
query = words_df \
    .writeStream \
    .outputMode("append") \
    .format("kafka") \
    .option("kafka.bootstrap.servers", kafka_bootstrap_servers) \
    .option("topic", kafka_write_topic) \
    .option("checkpointLocation", checkpoint_location) \
    .start()

In [ ]:
# Start streaming data to topic2
query.awaitTermination()